In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import random
import twitter
import emoji
import itertools
import pandas as pd
from itertools import chain
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
import keras.callbacks
import json

import os
# import nb_utils
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Embedding, GlobalMaxPooling1D
from keras.models import Model
from keras.layers import Concatenate, Average

from gensim.models import Word2Vec

Using TensorFlow backend.


In [0]:
!pip install twitter

     |████████████████████████████████| 61kB 3.7MB/s 


In [0]:
!pip install emoji

     |████████████████████████████████| 51kB 3.1MB/s 
  Created wheel for emoji: filename=emoji-0.5.4-cp36-none-any.whl size=42175 sha256=f115e3f3affbd5465f797069c4e04dfc4287763494ea53593ac65ef532b738a0
  Stored in directory: /root/.cache/pip/wheels/2a/a9/0a/4f8e8cce8074232aba240caca3fade315bb49fac68808d1a9c
Successfully built emoji


In [0]:
CONSUMER_KEY = 'xbMuxcJpRTiVGt2C2EYnA'
CONSUMER_SECRET = '2DbQTsvIptkPTdaUcos8DDvQH9fzO0hNjJpUT2uVzQ'
ACCESS_TOKEN = '7319442-EDm4CPxL7W4KkZcGWRMJNVHp88W5OH9vgblu898fg'
ACCESS_SECRET = '5ZxJSbqXhG7uhgXzTFWf9XhkfsxxinlPRXyDTzbA9w'

In [0]:
auth=twitter.OAuth(
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    token=ACCESS_TOKEN,
    token_secret=ACCESS_SECRET,
)

status_stream = twitter.TwitterStream(auth=auth).statuses

[x['text'] for x in itertools.islice(status_stream.sample(), 0, 5) if x.get('text')]

['يارب باعد بيني وبين الهياط مو ناكبني الا الهياط مسويه اني مدركه نص السلايدات',
 '@Wf1234123 💙💙💙',
 '@TcraiScanner Direi che è proprio una battuta da cinepanettone scadente e scaduto XD',
 '両思いなんだけどお互い付き合えないって場合どうすればいいんでしょうか？？',
 'RT @JMcintosh91: @KTHopkins Import the 3rd world, become the 3rd world.']

In [0]:
status_stream = twitter.TwitterStream(auth=auth).statuses

def english_has_emoji(tweet):
    if tweet.get('lang') != 'en':
        return False
    return any(ch for ch in tweet.get('text', '') if ch in emoji.UNICODE_EMOJI)

%time tweets = list(itertools.islice(filter(english_has_emoji, status_stream.sample()), 0, 100))

CPU times: user 1.04 s, sys: 93.4 ms, total: 1.13 s
Wall time: 56.6 s


In [0]:
stripped = []
for tweet in tweets:
    text = tweet['text']
    emojis = {ch for ch in text if ch in emoji.UNICODE_EMOJI}
    if len(emojis) == 1:
        emoiji = emojis.pop()
        text = ''.join(ch for ch in text if ch != emoiji)
        stripped.append((text, emoiji))
len(stripped)

64

Using the CNN

In [0]:
all_tweets = pd.read_csv('/content/drive/My Drive/data/emojis.csv')
all_tweets['emoji'].value_counts()

😂    124823
❤     43218
😍     40566
😭     35714
😊     20076
      ...  
🦈         1
📳         1
🔀         1
🕜         1
📭         1
Name: emoji, Length: 989, dtype: int64

In [0]:
tweets = all_tweets.groupby('emoji').filter(lambda c:len(c) > 1000)
tweets['emoji'].value_counts()

😂    124823
❤     43218
😍     40566
😭     35714
😊     20076
      ...  
😥      1072
🙁      1066
🤕      1065
😰      1013
☀      1013
Name: emoji, Length: 121, dtype: int64

In [0]:
max(tweets['text'], key=lambda t:len(t))

"Don't worry, my love, I don't get you wrong Don't get me wrong. We're connected with our hearts, souls, minds, bodies. If any doubt, we ask"

In [0]:
chars = list(sorted(set(chain(*tweets['text']))))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
max_sequence_len = max(len(x) for x in tweets['text'])

emojis = list(sorted(set(tweets['emoji'])))
emoji_to_idx = {em: idx for idx, em in enumerate(emojis)}
emojis[:10]

train_tweets, test_tweets = train_test_split(tweets, test_size=0.1)


In [0]:
def data_generator(tweets, batch_size):
    while True:
        if batch_size is None:
            batch = tweets
            batch_size = batch.shape[0]
        else:
            batch = tweets.sample(batch_size)
        X = np.zeros((batch_size, max_sequence_len, len(chars)))
        y = np.zeros((batch_size,))
        for row_idx, (_, row) in enumerate(batch.iterrows()):
            y[row_idx] = emoji_to_idx[row['emoji']]
            for ch_idx, ch in enumerate(row['text']):
                X[row_idx, ch_idx, char_to_idx[ch]] = 1
        yield X, y

next(data_generator(tweets, 10))

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],


In [0]:
def create_char_cnn_model(num_chars, max_sequence_len, num_labels):
    char_input = Input(shape=(max_sequence_len, num_chars), name='char_cnn_input')
    
    conv_1x = Conv1D(128, 6, activation='relu', padding='valid')(char_input)
    max_pool_1x = MaxPooling1D(4)(conv_1x)
    conv_2x = Conv1D(256, 6, activation='relu', padding='valid')(max_pool_1x)
    max_pool_2x = MaxPooling1D(4)(conv_2x)

    flatten = Flatten()(max_pool_2x)
    dense = Dense(128, activation='relu')(flatten)
    preds = Dense(num_labels, activation='softmax', name='char_cnn_predictions')(dense)

    model = Model(char_input, preds)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['acc'])
    return model

char_cnn_model = create_char_cnn_model(len(char_to_idx), max_sequence_len, len(emojis))
char_cnn_model.summary()







Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_cnn_input (InputLayer)  (None, 139, 96)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 134, 128)          73856     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 33, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 28, 256)           196864    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7, 256)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1792)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)             

In [0]:
len(train_tweets)

634695

In [0]:

early = keras.callbacks.EarlyStopping(monitor='loss',
                              min_delta=0.03,
                              patience=2,
                              verbose=0, mode='auto')

BATCH_SIZE = 1024
char_cnn_model.fit_generator(
    data_generator(train_tweets, batch_size=BATCH_SIZE),
    epochs=20,
    steps_per_epoch=len(train_tweets)/BATCH_SIZE,
    verbose=2,
    callbacks=[early]
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20







KeyboardInterrupt: ignored

In [0]:
char_cnn_model.save("/content/drive/My Drive/data/cnn.h5")

In [0]:
from keras.models import load_model
model=load_model("/content/drive/My Drive/data/lstm.h5")

In [0]:
from keras.models import load_model
model=load_model("/content/drive/My Drive/data/cnn.h5")

In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
char_cnn_input (InputLayer)  (None, 139, 96)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 134, 128)          73856     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 33, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 28, 256)           196864    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 7, 256)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1792)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               2295

In [0]:
from keras.models import load_model
model_lstm=load_model("/content/drive/My Drive/data/lstm.h5")

In [0]:
model_lstm.summary();

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_input (InputLayer)      (None, None)              0         
_________________________________________________________________
lstm_embedding (Embedding)   (None, None, 100)         5000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               117248    
_________________________________________________________________
lstm_predictions (Dense)     (None, 121)               15609     
Total params: 5,132,857
Trainable params: 5,132,857
Non-trainable params: 0
_________________________________________________________________


In [0]:
VOCAB_SIZE = 50000
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(tweets['text'])
training_tokens = tokenizer.texts_to_sequences(train_tweets['text'])
test_tokens = tokenizer.texts_to_sequences(test_tweets['text'])
max_num_tokens = max(len(x) for x in chain(training_tokens, test_tokens))
training_tokens = pad_sequences(training_tokens, maxlen=max_num_tokens)
test_tokens = pad_sequences(test_tokens, maxlen=max_num_tokens)

In [0]:
test_char_vectors, _ = next(data_generator(test_tweets, None))

In [0]:
def load_weights(tokenizer):
    model = Word2Vec.load('/content/drive/My Drive/data/twitter_stream_w2v.model')
    w2v = np.zeros((tokenizer.num_words, w2v_model.syn0.shape[1]))
    for k, v in tokenizer.word_index.items():
        if v >= tokenizer.num_words:
            continue
        if k in w2v_model:
            w2v[v] = w2v_model[k]
    return w2v
model = Word2Vec.load('/content/drive/My Drive/data/twitter_stream_w2v.model')
w2v = np.zeros((tokenizer.num_words, model.wv.syn0.shape[1]))
found = 0
for k, v in tokenizer.word_index.items():
    if v >= tokenizer.num_words:
        continue
    if k in model:
        w2v[v] = model[k]
        found += 1
found, tokenizer.num_words

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(17267, 50000)

In [0]:
predictions = {
    label: [emojis[np.argmax(x)] for x in pred]
    for label, pred in (
        ('lstm', model_lstm.predict(test_tokens[:1000])),
        ('char_cnn', model.predict(test_char_vectors[:1000])),
        
    )
}

In [0]:
pd.options.display.max_colwidth = 128
test_df = test_tweets[:1000].reset_index()
eval_df = pd.DataFrame({
    'content': test_df['text'],
    'true': test_df['emoji'],
    **predictions
})
eval_df[['content', 'true', 'char_cnn', 'lstm']].head(25)

,content,true,char_cnn,lstm
0,i love being sick on game days,😩,😩,😂
1,@_white_girl_xo: Death becomes of those who make lovely things like this. @nt_losechesters,😭,😭,😂
2,@malikofori: This is still the funniest story on twitter.,😂,😂,😂
3,Thank you so much PICE AggregatesRailsACI. Lavern Civil,😊,❤,😊
4,@Rafurl: Laughs infinixically,😂,😂,😂
5,@acidhunk: goodmorning today feels like a perfect day to have a mental breakdown,😊,😊,😊
6,"@camerondallas: RT this & Tweet "" I vote @camerondallas for Entertainer of The Year #Streamys"" for a follow",😊,😊,😊
7,@valivy1: Bye Elnella ... We r behind u all way!!!,❤,❤,❤
8,Omg it's lit @FrankArensMusic,😍,😂,🔥
9,"You dont have any right to call me fat, Coz in the first place, I didn't even call you ugly!!! #ayawkogbiklaLage! #malditako!",😈,😈,😂


In [0]:

emotion_df['sentiment'].value_counts()

neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: sentiment, dtype: int64

In [0]:
import pandas as pd
from keras.utils.data_utils import get_file


emotion_csv = get_file('text_emotion.csv', 
                       'https://www.crowdflower.com/wp-content/uploads/2016/07/text_emotion.csv')
emotion_df = pd.read_csv(emotion_csv)

In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot

VOCAB_SIZE = 50000
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(emotion_df['content'])

In [0]:
import os
import re
import numpy as np
import gensim

CACHE_DIR = os.path.expanduser('~/.cache/dl-cookbook')

def download(url):
    filename = os.path.join(CACHE_DIR, re.sub('[^a-zA-Z0-9.]+', '_', url))
    if os.path.exists(filename):
        return filename
    else:
        os.system('mkdir -p "%s"' % CACHE_DIR)
        assert os.system('wget -O "%s" "%s"' % (filename, url)) == 0
        return filename
    
    
def load_w2v(tokenizer=None):
    word2vec_gz = download('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz')
    word2vec_vectors = word2vec_gz.replace('.gz', '')
    if not os.path.exists(word2vec_vectors):
        assert os.system('gunzip -d --keep "%s"' % word2vec_gz) == 0
        
    w2v_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_vectors, binary=True)
    
    total_count = sum(tokenizer.word_counts.values())
    idf_dict = { k: np.log(total_count/v) for (k,v) in tokenizer.word_counts.items() }
    
    w2v = np.zeros((tokenizer.num_words, w2v_model.syn0.shape[1]))
    idf = np.zeros((tokenizer.num_words, 1))

    for k, v in tokenizer.word_index.items():
        if v >= tokenizer.num_words:
            continue

        if k in w2v_model:
            w2v[v] = w2v_model[k]
            idf[v] = idf_dict[k]

    del w2v_model
    return w2v, idf

In [0]:
w2v, idf = load_w2v(tokenizer)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).


In [0]:
from itertools import chain
from keras.preprocessing.sequence import pad_sequences
import numpy as np

chars = list(sorted(set(chain(*emotion_df['content']))))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
max_sequence_len = max(len(x) for x in emotion_df['content'])

char_vectors = []
for txt in emotion_df['content']:
    vec = np.zeros((max_sequence_len, len(char_to_idx)))
    vec[np.arange(len(txt)), [char_to_idx[ch] for ch in txt]] = 1
    char_vectors.append(vec)
char_vectors = np.asarray(char_vectors)
char_vectors = pad_sequences(char_vectors)
labels = label_encoder.transform(emotion_df['sentiment'])


def split(lst):
    training_count = int(0.9 * len(char_vectors))
    return lst[:training_count], lst[training_count:]

training_char_vectors, test_char_vectors = split(char_vectors)
training_labels, test_labels = split(labels)

char_vectors.shape

In [0]:
tokens = tokenizer.texts_to_sequences(emotion_df['content'])
tokens = pad_sequences(tokens)



training_tokens, training_labels = tokens[:training_count], labels[:training_count]
test_tokens, test_labels = tokens[training_count:], labels[training_count:]

In [0]:
from keras import layers, models
import keras.backend as K


def make_embedding(name, vocab_size, embedding_size, weights=None, mask_zero=True):
    if weights is not None:
        return layers.Embedding(mask_zero=mask_zero, input_dim=vocab_size, 
                                output_dim=weights.shape[1], 
                                weights=[weights], trainable=False, 
                                name='%s/embedding' % name)
    else:
        return layers.Embedding(mask_zero=mask_zero, input_dim=vocab_size, 
                                output_dim=embedding_size,
                                name='%s/embedding' % name)

def create_unigram_model(vocab_size, embedding_size=None, embedding_weights=None, idf_weights=None):
    assert not (embedding_size is None and embedding_weights is None)
    message = layers.Input(shape=(None,), dtype='int32', name='message')
    
    embedding = make_embedding('message_vec', vocab_size, embedding_size, embedding_weights)
    idf = make_embedding('message_idf', vocab_size, embedding_size, idf_weights)

    mask = layers.Masking(mask_value=0)
    def _combine_and_sum(args):
        embedding, idf = args
        return K.sum(embedding * K.abs(idf), axis=1)

    sum_layer = layers.Lambda(_combine_and_sum, name='combine_and_sum')
    sum_msg = sum_layer([mask(embedding(message)), idf(message)])
    fc1 = layers.Dense(units=128, activation='relu')(sum_msg)
    categories = layers.Dense(units=len(label_encoder.classes_), activation='softmax')(fc1)
    
    model = models.Model(
        inputs=[message],
        outputs=categories,
    )
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model.summary()
    return model

unigram_model = create_unigram_model(vocab_size=VOCAB_SIZE,
                                     embedding_weights=w2v,
                                     idf_weights=idf)

NameError: ignored

In [0]:
from itertools import chain
from keras.preprocessing.sequence import pad_sequences
import numpy as np

chars = list(sorted(set(chain(*emotion_df['content']))))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
max_sequence_len = max(len(x) for x in emotion_df['content'])

char_vectors = []
for txt in emotion_df['content']:
    vec = np.zeros((max_sequence_len, len(char_to_idx)))
    vec[np.arange(len(txt)), [char_to_idx[ch] for ch in txt]] = 1
    char_vectors.append(vec)
char_vectors = np.asarray(char_vectors)
char_vectors = pad_sequences(char_vectors)
labels = label_encoder.transform(emotion_df['sentiment'])


def split(lst):
    training_count = int(0.9 * len(char_vectors))
    return lst[:training_count], lst[training_count:]

training_char_vectors, test_char_vectors = split(char_vectors)
training_labels, test_labels = split(labels)

char_vectors.shape

(40000, 167, 100)

In [0]:
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

VOCAB_SIZE = 50000

tfidf_vec = TfidfVectorizer(max_features=VOCAB_SIZE)
label_encoder = LabelEncoder()

X = tfidf_vec.fit_transform(emotion_df['content'])
y = label_encoder.fit_transform(emotion_df['sentiment'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)